# Evaluation

> Scripts to perform evaluation on the data

In [1]:
#| default_exp evaluation

In [2]:
#| export
#| hide
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score, v_measure_score, fowlkes_mallows_score, silhouette_score, jaccard_score, confusion_matrix
from sklearn.metrics.cluster import contingency_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import squareform
from fastdtw import fastdtw
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import cKDTree

Auxiliar Functions

In [3]:
#| export
def plot_metric_heatmaps(results, distance_metrics, clustering_algorithms, evaluation_metrics):
    """
    Plot heatmaps for each evaluation metric.
    """
    n_metrics = len(evaluation_metrics)
    n_cols = 3
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    axes = axes.flatten()
    
    for i, metric in enumerate(evaluation_metrics):
        data = np.zeros((len(distance_metrics), len(clustering_algorithms)))
        for d, distance in enumerate(distance_metrics):
            for c, clustering in enumerate(clustering_algorithms):
                data[d, c] = results[f"{distance}_{clustering}"][metric]
        
        sns.heatmap(data, annot=True, fmt=".2f", cmap="YlGnBu", 
                    xticklabels=clustering_algorithms, yticklabels=distance_metrics, ax=axes[i])
        axes[i].set_title(f"{metric} Scores")
        axes[i].set_xlabel("Clustering Algorithms")
        axes[i].set_ylabel("Distance Metrics")
    
    # Remove any unused subplots
    for i in range(n_metrics, len(axes)):
        fig.delaxes(axes[i])
    
    plt.tight_layout()
    plt.show()

In [4]:
#| export
def plot_comparison(orbit_df, synthetic_orbit_df):
    """
    Function to create a scatter plot comparing 'period' and 'calculated_jacobi'
    between two DataFrames and plot the index of points in synthetic_orbit_df.
    
    Parameters:
    - orbit_df: DataFrame containing 'period' and 'calculated_jacobi' columns
    - synthetic_orbit_df: DataFrame containing 'period' and 'calculated_jacobi' columns
    """
    plt.figure(figsize=(10, 6))

    # Plot for orbit_df
    plt.scatter(orbit_df['period'], orbit_df['calculated_jacobi'], color='blue', label='orbit_df', marker='o')

    # Plot for synthetic_orbit_df
    plt.scatter(synthetic_orbit_df['period'], synthetic_orbit_df['calculated_jacobi'], color='red', label='synthetic_orbit_df', marker='x')

    # Annotate the points in synthetic_orbit_df with their indices
    for i in synthetic_orbit_df.index:
        plt.text(synthetic_orbit_df['period'].loc[i], 
                 synthetic_orbit_df['calculated_jacobi'].loc[i], 
                 str(int(i)), 
                 fontsize=9, color='black', ha='right')

    # Add labels and title
    plt.xlabel('Period')
    plt.ylabel('Calculated Jacobi')
    plt.title('Comparison of Features Between Two DataFrames')
    plt.legend()  # Add legend to distinguish between the groups

    # Display the plot
    plt.grid(True)
    plt.show()

def calculate_closest_feature_distances(orbit_df, synthetic_orbit_df, features, plot_comparison=True):
    """
    Calculate the distance from each point in synthetic_orbit_df to the closest point in orbit_df
    based on specified features.
    
    Parameters:
    - orbit_df: DataFrame containing the training data.
    - synthetic_orbit_df: DataFrame containing the synthetic data.
    - features: List of features (columns) to use for calculating the distances.
    
    Returns:
    - distances: A NumPy array of the minimum distances from each synthetic point to the nearest orbit point.
    """
    # Extract the relevant features from both DataFrames
    orbit_points = orbit_df[features].values
    synthetic_points = synthetic_orbit_df[features].values

    # Create a KDTree for efficient nearest-neighbor search in orbit_df
    tree = cKDTree(orbit_points)

    # Query the KDTree with the synthetic points to find the distance to the nearest orbit point
    distances, _ = tree.query(synthetic_points, k=1)

    if plot_comparison:
        plot_comparison(orbit_df, synthetic_orbit_df)

    return distances

In [5]:
#| export
def find_non_matching_elements(main_array, check_array):
    """
    Finds elements in check_array that are not present in main_array.

    Parameters:
    main_array (numpy.ndarray): The main array with larger set of elements.
    check_array (numpy.ndarray): The array with elements to check against the main array.

    Returns:
    numpy.ndarray: Elements in check_array that are not in main_array.
    """
    # Convert arrays to tuples to enable comparison
    main_set = set(map(tuple, main_array))
    check_set = set(map(tuple, check_array))

    # Find elements in check_set that are not in main_set
    non_matching_elements = check_set - main_set

    # Convert the result back to a numpy array
    non_matching_elements_array = np.array(list(non_matching_elements))

    return non_matching_elements_array

## Evaluate Clustering with Multiple Labels

In [6]:
#| export
def evaluate_clustering_multiple_labels(latent_representations: np.ndarray,  # The latent space data.
                                        list_of_labels: list,                # List of true labels or a single true labels array.
                                        clustering_method: str = 'kmeans',   # The clustering algorithm to use ('kmeans', 'gmm', 'dbscan').
                                        label_names: list = None,            # Optional names for the label sets.
                                        **kwargs                             # Additional arguments for the clustering algorithm.
                                       ) -> dict:                            # Returns a dictionary with clustering metrics.
    """
    Evaluates the clustering quality of the latent representations for one or multiple sets of labels.
    """
    
    def calculate_clustering_accuracy(true_labels, pred_labels):
        contingency = contingency_matrix(true_labels, pred_labels)
        row_ind, col_ind = linear_sum_assignment(-contingency)
        return contingency[row_ind, col_ind].sum() / np.sum(contingency)

    # Ensure list_of_labels is a list of arrays
    if isinstance(list_of_labels, np.ndarray):
        list_of_labels = [list_of_labels]
    
    # Use default names if label_names are not provided
    if label_names is None:
        label_names = [f'Set_{i+1}' for i in range(len(list_of_labels))]
    
    combined_metrics = {}
    average_metrics = {
        'ARI': 0,
        'NMI': 0,
        'Homogeneity': 0,
        'Completeness': 0,
        'V-Measure': 0,
        'FMI': 0,
        'Purity': 0,
        'Silhouette Score': 0,
        'Jaccard': 0,
        'Accuracy': 0
    }
    num_label_sets = len(list_of_labels)
    
    label_encoders = [LabelEncoder() for _ in range(num_label_sets)]
    encoded_labels_list = [label_encoders[i].fit_transform(list_of_labels[i]) for i in range(num_label_sets)]
    
    for i, true_labels in enumerate(encoded_labels_list):
        # Determine the number of clusters
        n_clusters = len(np.unique(true_labels))
        
        # Apply the selected clustering algorithm
        if clustering_method == 'kmeans':
            clusterer = KMeans(n_clusters=n_clusters, random_state=42, n_init=10, **kwargs)
            pred_labels = clusterer.fit_predict(latent_representations)
        elif clustering_method == 'gmm':
            clusterer = GaussianMixture(n_components=n_clusters, random_state=42, **kwargs)
            pred_labels = clusterer.fit_predict(latent_representations)
        elif clustering_method == 'dbscan':
            clusterer = DBSCAN(**kwargs)
            pred_labels = clusterer.fit_predict(latent_representations)
        else:
            raise ValueError("Unsupported clustering method. Choose from 'kmeans', 'gmm', 'dbscan'.")
        
        # Calculate clustering metrics
        ari = adjusted_rand_score(true_labels, pred_labels)
        nmi = normalized_mutual_info_score(true_labels, pred_labels)
        homogeneity = homogeneity_score(true_labels, pred_labels)
        completeness = completeness_score(true_labels, pred_labels)
        v_measure = v_measure_score(true_labels, pred_labels)
        fmi = fowlkes_mallows_score(true_labels, pred_labels)
        
        # Purity
        cont_matrix = contingency_matrix(true_labels, pred_labels)
        purity = np.sum(np.amax(cont_matrix, axis=0)) / np.sum(cont_matrix)
        
        # Silhouette Score
        silhouette = silhouette_score(latent_representations, pred_labels)
        
        # Jaccard Coefficient and Accuracy
        jaccard = jaccard_score(true_labels, pred_labels, average='macro')
        accuracy = calculate_clustering_accuracy(true_labels, pred_labels)
        
        # Store the results for this set of labels
        combined_metrics.update({
            f'{label_names[i]}_ari': ari,
            f'{label_names[i]}_nmi': nmi,
            f'{label_names[i]}_homogeneity': homogeneity,
            f'{label_names[i]}_completeness': completeness,
            f'{label_names[i]}_v-measure': v_measure,
            f'{label_names[i]}_fmi': fmi,
            f'{label_names[i]}_purity': purity,
            f'{label_names[i]}_silhouette_score': silhouette,
            f'{label_names[i]}_jaccard': jaccard,
            f'{label_names[i]}_accuracy': accuracy
        })
        
        # Accumulate the results for averaging
        average_metrics['ARI'] += ari
        average_metrics['NMI'] += nmi
        average_metrics['Homogeneity'] += homogeneity
        average_metrics['Completeness'] += completeness
        average_metrics['V-Measure'] += v_measure
        average_metrics['FMI'] += fmi
        average_metrics['Purity'] += purity
        average_metrics['Silhouette Score'] += silhouette
        average_metrics['Jaccard'] += jaccard
        average_metrics['Accuracy'] += accuracy
    
    # Compute the average metrics if there are multiple sets of labels
    if num_label_sets > 1:
        for key in average_metrics:
            combined_metrics[f'average_{key}'] = average_metrics[key] / num_label_sets
    
    return combined_metrics

## Physical Distances

In [7]:
#| export
def euclidean_distance(x):
    n = x.shape[0]
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist = np.sqrt(np.sum((x[i] - x[j])**2))
            dist_matrix[i, j] = dist_matrix[j, i] = dist
    return dist_matrix

In [8]:
#| export
def manhattan_distance(x):
    n = x.shape[0]
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist = np.sum(np.abs(x[i] - x[j]))
            dist_matrix[i, j] = dist_matrix[j, i] = dist
    return dist_matrix

In [9]:
#| export
def cosine_distance(x):
    n = x.shape[0]
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dot_product = np.sum(x[i] * x[j])
            norm_i = np.sqrt(np.sum(x[i]**2))
            norm_j = np.sqrt(np.sum(x[j]**2))
            similarity = dot_product / (norm_i * norm_j)
            dist = 1 - similarity  # This ensures the distance is between 0 and 2
            dist_matrix[i, j] = dist_matrix[j, i] = dist
    return dist_matrix

In [10]:
#| export
def dtw_distance(x):
    n = x.shape[0]
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance, _ = fastdtw(x[i].T, x[j].T)
            dist_matrix[i, j] = dist_matrix[j, i] = distance
    return dist_matrix

### Get the Closest Orbits

In [11]:
#| export
def find_nearest_orbits(single_orbit: np.ndarray,
                        orbit_data: np.ndarray,
                        n: int,
                        distance_metric: str = 'euclidean') -> np.ndarray:
    """
    Finds the n closest orbits in orbit_data to the single_orbit based on the specified distance metric.
    
    :param single_orbit: The reference orbit (shape: [n_features, n_time_steps]).
    :param orbit_data: The dataset of orbits (shape: [n_samples, n_features, n_time_steps]).
    :param n: The number of closest orbits to return.
    :param distance_metric: The distance metric to use ('euclidean', 'manhattan', 'cosine', 'dtw').
                            Defaults to 'euclidean'.
    :return: Indices of the n closest orbits in orbit_data.
    """
    
    # Stack the single_orbit with the orbit_data to form a new dataset
    # This allows us to compute the distance matrix including the single_orbit
    combined_data = np.vstack([orbit_data, single_orbit[np.newaxis, ...]])
    
    # Select the appropriate distance function
    if distance_metric == 'euclidean':
        distance_matrix = euclidean_distance(combined_data)
    elif distance_metric == 'manhattan':
        distance_matrix = manhattan_distance(combined_data)
    elif distance_metric == 'cosine':
        distance_matrix = cosine_distance(combined_data)
    elif distance_metric == 'dtw':
        distance_matrix = dtw_distance(combined_data)
    else:
        raise ValueError(f"Unsupported distance metric: {distance_metric}")
    
    # The single_orbit is the last entry in combined_data
    # Extract distances from the last row (or column) excluding the distance to itself
    distances = distance_matrix[-1, :-1]
    
    # Get the indices of the n smallest distances
    nearest_indices = np.argsort(distances)[:n]
    
    return nearest_indices

## Evaluate Distance Metrics

In [12]:
#| export
def evaluate_distance_metrics_and_clustering(orbit_data: np.ndarray,
                                    true_labels: np.ndarray,
                                    distance_metrics: list = None,
                                    clustering_algorithms: list = None,
                                    evaluation_metrics: list = None,
                                    n_clusters: int = None,
                                    plot_results: bool = True):
    """
    Evaluates specified distance metrics and clustering algorithms on the given orbit data.
    
    :param orbit_data: The orbit data as a multivariate time series (shape: [n_samples, n_features, n_time_steps]).
    :param true_labels: Array of true labels for the orbit data.
    :param distance_metrics: List of strings specifying distance metrics to use. If None, all available metrics are used.
    :param clustering_algorithms: List of strings specifying clustering algorithms to use. If None, all available algorithms are used.
    :param evaluation_metrics: List of strings specifying evaluation metrics to use. If None, all available metrics are used.
    :param n_clusters: Number of clusters for algorithms that require it. If None, it will be inferred from labels.
    :param plot_results: If True, plot heatmaps of the results.
    :return: A dictionary containing results for all combinations of metrics and clustering algorithms.
    """

    def calculate_clustering_accuracy(true_labels, pred_labels):
        cm = confusion_matrix(true_labels, pred_labels)
        row_ind, col_ind = linear_sum_assignment(-cm)
        return cm[row_ind, col_ind].sum() / np.sum(cm)

    # Define available distance metrics
    available_distance_metrics = {
        'euclidean': euclidean_distance,
        'manhattan': manhattan_distance,
        'cosine': cosine_distance,
        'dtw': dtw_distance
    }
    
    # Define available clustering algorithms
    available_clustering_algorithms = {
        'kmeans': KMeans,
        'gmm': GaussianMixture,
        'dbscan': DBSCAN
    }
    
    # Define available evaluation metrics
    available_evaluation_metrics = {
        'ARI': adjusted_rand_score,
        'NMI': normalized_mutual_info_score,
        'Homogeneity': homogeneity_score,
        'Completeness': completeness_score,
        'V-Measure': v_measure_score,
        'Purity': lambda true, pred: np.sum(np.amax(confusion_matrix(true, pred), axis=0)) / np.sum(confusion_matrix(true, pred)),
        'Accuracy': calculate_clustering_accuracy
    }
    
    # If no metrics specified, use all available
    if distance_metrics is None:
        distance_metrics = list(available_distance_metrics.keys())
    
    # If no algorithms specified, use all available
    if clustering_algorithms is None:
        clustering_algorithms = list(available_clustering_algorithms.keys())
    
    # If no evaluation metrics specified, use all available
    if evaluation_metrics is None:
        evaluation_metrics = list(available_evaluation_metrics.keys())
    
    results = {}
    
    # If n_clusters is not provided, infer it from the labels
    if n_clusters is None:
        n_clusters = len(np.unique(true_labels))
    
    for distance_metric in distance_metrics:
        if distance_metric not in available_distance_metrics:
            print(f"Warning: {distance_metric} is not an available distance metric. Skipping.")
            continue
        
        print(f"Computing {distance_metric} distances...")
        metric_func = available_distance_metrics[distance_metric]
        distance_matrix = metric_func(orbit_data)

        if np.any(distance_matrix < 0):
            distance_matrix = distance_matrix - np.min(distance_matrix)
            np.fill_diagonal(distance_matrix, 0)
        
        for algo_name in clustering_algorithms:
            if algo_name not in available_clustering_algorithms:
                print(f"Warning: {algo_name} is not an available clustering algorithm. Skipping.")
                continue
            
            print(f"Clustering with {algo_name}...")
            algo_class = available_clustering_algorithms[algo_name]
            
            if algo_name == 'dbscan':
                # For DBSCAN, we need to estimate eps
                distances = squareform(distance_matrix)
                eps = np.percentile(distances, 10)  # Use the 10th percentile of distances as eps
                clusterer = algo_class(eps=eps, min_samples=5, metric='precomputed')
                labels = clusterer.fit_predict(distance_matrix)
            elif algo_name == 'kmeans':
                clusterer = algo_class(n_clusters=n_clusters, random_state=42)
                labels = clusterer.fit_predict(distance_matrix)
            elif algo_name == 'gmm':
                clusterer = algo_class(n_components=n_clusters, random_state=42)
                labels = clusterer.fit_predict(distance_matrix)
            else:
                clusterer = algo_class()  # Use default parameters for other algorithms
                labels = clusterer.fit_predict(distance_matrix)
            
            # Evaluate clustering
            eval_results = {}
            for eval_metric in evaluation_metrics:
                if eval_metric not in available_evaluation_metrics:
                    print(f"Warning: {eval_metric} is not an available evaluation metric. Skipping.")
                    continue
                else:
                    eval_results[eval_metric] = available_evaluation_metrics[eval_metric](true_labels, labels)
            
            # Store results
            results[f"{distance_metric}_{algo_name}"] = eval_results
    
    if plot_results:
        plot_metric_heatmaps(results, distance_metrics, clustering_algorithms, evaluation_metrics)
    
    return results



In [13]:
#| hide
import nbdev; nbdev.nbdev_export()